In [1]:
%matplotlib inline 

import requests 
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operations
#from __future__ import division
from math import log10,exp
import seaborn as sns
import scipy as sp
import sklearn
import sklearn.model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import csv

pd.options.display.mpl_style = 'default'

C:\Users\User\AppData\Local\conda\conda\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py:3267: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
import warnings
import itertools
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import matplotlib

In [3]:
# load the dataset

dataload_num = 1
if dataload_num == 1:
    data = pd.read_csv('Solar/Home-electricity-data-v3.csv')
#end

In [ ]:
# Choose the data from household with certain post codes, i.e., 2259, 2261, ......
data_Postcode2259=data[data.Postcode == 2259]
data_Postcode2261=data[data.Postcode == 2261]
data_Postcode2262=data[data.Postcode == 2262]
data_Postcode2263=data[data.Postcode == 2263]
data_All4Postcodes=pd.concat([data_Postcode2259, data_Postcode2261,data_Postcode2262,data_Postcode2263], ignore_index=True)
#df.groupby(['Postcode', 'Customer'])
print(data_All4Postcodes.groupby(['Postcode', 'Customer']).count())

In [ ]:
# Adding day of week to separate weekday and weekend
import datetime as dt

data_All4Postcodes.date = data_All4Postcodes.date.apply(lambda x: dt.datetime.strptime(x, '%d/%m/%Y'))
data_All4Postcodes['day_of_week'] = data_All4Postcodes['date'].dt.dayofweek
data_All4Postcodes

In [ ]:
data_All4Postcodes_Consumption_GC=data_All4Postcodes[data_All4Postcodes.Consumption_Category == 'GC']
data_All4Postcodes_Consumption_GG=data_All4Postcodes[data_All4Postcodes.Consumption_Category == 'GG']
data_All4Postcodes_Consumption_GC

In [ ]:
HalfHourTimestamp = list(data_All4Postcodes_Consumption_GC.columns)[5:53]

Bin_Positions=[1,2864,5728,8592,11457,14321,17185,20050,22914]
Bin_Range_Values=[]
for x in range (48):
    data_All4Postcodes_Consumption_GC_temp=data_All4Postcodes_Consumption_GC.loc[:,HalfHourTimestamp[x]]
    Bin_Range_Values_tmp = [data_All4Postcodes_Consumption_GC_temp.nsmallest(Bin_Positions[y]).iloc[-1] for y in range (9)]
    Bin_Range_Values.append(Bin_Range_Values_tmp)
        #df11x2=df11.loc[:,'6:00']
        #df11x2r[x,y]=df11x2.nsmallest(pos[y]).iloc[-1]
Bin_Range_Values

In [ ]:
with open('Bin_Range_Values_halfhourlyv8.txt', 'w') as f:
    for item in Bin_Range_Values:
        f.write("%s\n" % item)

In [ ]:
DateTimestamp = pd.date_range(start = '2012-07-01', end = '2013-06-30', freq ="D")
dACG=data_All4Postcodes_Consumption_GC
dACG_Date=dACG[dACG.date == '2012-07-01']
dACG_Date_Time=dACG_Date.loc[:,'0:30']
dACG_Date_Time_Bins=dACG_Date_Time.groupby(pd.cut(dACG_Date_Time, Bin_Range_Values[0])).count().values.reshape(1, 8).T
dACG_Date_Time_Bins_temp=dACG_Date_Time_Bins
dACG_Date_Time_Bins_Noise= dACG_Date_Time_Bins + np.random.laplace(loc=0.0, scale=1.0, size=dACG_Date_Time_Bins.shape)
dACG_Date_Time_Bins_Noise=np.around((dACG_Date_Time_Bins_Noise/np.sum(dACG_Date_Time_Bins_Noise))*np.sum(dACG_Date_Time_Bins))
dACG_Date_Time_Bins_Noise_temp=dACG_Date_Time_Bins_Noise
dACG_Date_Time_Bins_Noise_temp
dACG_Date_Time_Bined_Accu_Elec=dACG_Date_Time_Bins[0]*(Bin_Range_Values[0][1]+Bin_Range_Values[0][0])/2+dACG_Date_Time_Bins[1]*(Bin_Range_Values[0][2]+Bin_Range_Values[0][1])/2+dACG_Date_Time_Bins[2]*(Bin_Range_Values[0][3]+Bin_Range_Values[0][2])/2+dACG_Date_Time_Bins[3]*(Bin_Range_Values[0][4]+Bin_Range_Values[0][3])/2+dACG_Date_Time_Bins[4]*(Bin_Range_Values[0][5]+Bin_Range_Values[0][4])/2+dACG_Date_Time_Bins[5]*(Bin_Range_Values[0][6]+Bin_Range_Values[0][5])/2+dACG_Date_Time_Bins[6]*(Bin_Range_Values[0][7]+Bin_Range_Values[0][6])/2+dACG_Date_Time_Bins[7]*(Bin_Range_Values[0][8]+Bin_Range_Values[0][7])/4
dACG_Date_Time_Bined_Noise_Accu_Elec=dACG_Date_Time_Bins_Noise[0]*(Bin_Range_Values[0][1]+Bin_Range_Values[0][0])/2+dACG_Date_Time_Bins_Noise[1]*(Bin_Range_Values[0][2]+Bin_Range_Values[0][1])/2+dACG_Date_Time_Bins_Noise[2]*(Bin_Range_Values[0][3]+Bin_Range_Values[0][2])/2+dACG_Date_Time_Bins_Noise[3]*(Bin_Range_Values[0][4]+Bin_Range_Values[0][3])/2+dACG_Date_Time_Bins_Noise[4]*(Bin_Range_Values[0][5]+Bin_Range_Values[0][4])/2+dACG_Date_Time_Bins_Noise[5]*(Bin_Range_Values[0][6]+Bin_Range_Values[0][5])/2+dACG_Date_Time_Bins_Noise[6]*(Bin_Range_Values[0][7]+Bin_Range_Values[0][6])/2+dACG_Date_Time_Bins_Noise[7]*(Bin_Range_Values[0][8]+Bin_Range_Values[0][7])/4
dACG_Date_Time_Bined_Accu_Elec_temp=dACG_Date_Time_Bined_Accu_Elec
dACG_Date_Time_Bined_Noise_Accu_Elec_temp=dACG_Date_Time_Bined_Noise_Accu_Elec
dACG_Date_Time_Bined_Noise_Accu_Elec_temp

In [ ]:

for x in DateTimestamp:
    dACG_Date=dACG[dACG.date == x]
    for y in range (48):
        #df11x2=df11.loc[:,HalfHourTimestamp1[x]]
        dACG_Date_Time=dACG_Date.loc[:,HalfHourTimestamp[y]]
        dACG_Date_Time_Bins=dACG_Date_Time.groupby(pd.cut(dACG_Date_Time, Bin_Range_Values[y])).count().values.reshape(1, 8).T

        dACG_Date_Time_Bins_Noise= dACG_Date_Time_Bins + np.random.laplace(loc=0.0, scale=1.0, size=dACG_Date_Time_Bins.shape)
        dACG_Date_Time_Bins_Noise=np.around((dACG_Date_Time_Bins_Noise/np.sum(dACG_Date_Time_Bins_Noise))*np.sum(dACG_Date_Time_Bins))

        dACG_Date_Time_Bined_Accu_Elec=dACG_Date_Time_Bins[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2+dACG_Date_Time_Bins[5]*(Bin_Range_Values[y][6]+Bin_Range_Values[y][5])/2+dACG_Date_Time_Bins[6]*(Bin_Range_Values[y][7]+Bin_Range_Values[y][6])/2+dACG_Date_Time_Bins[7]*(Bin_Range_Values[y][8]+Bin_Range_Values[y][7])/4
        dACG_Date_Time_Bined_Noise_Accu_Elec=dACG_Date_Time_Bins_Noise[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins_Noise[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins_Noise[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins_Noise[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins_Noise[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2+dACG_Date_Time_Bins_Noise[5]*(Bin_Range_Values[y][6]+Bin_Range_Values[y][5])/2+dACG_Date_Time_Bins_Noise[6]*(Bin_Range_Values[y][7]+Bin_Range_Values[y][6])/2+dACG_Date_Time_Bins_Noise[7]*(Bin_Range_Values[y][8]+Bin_Range_Values[y][7])/4

        #dACG_Date_Time_Bined_Accu_Elec=dACG_Date_Time_Bins[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2
        #dACG_Date_Time_Bined_Noise_Accu_Elec=dACG_Date_Time_Bins_Noise[0]*(Bin_Range_Values[y][1]+Bin_Range_Values[y][0])/2+dACG_Date_Time_Bins_Noise[1]*(Bin_Range_Values[y][2]+Bin_Range_Values[y][1])/2+dACG_Date_Time_Bins_Noise[2]*(Bin_Range_Values[y][3]+Bin_Range_Values[y][2])/2+dACG_Date_Time_Bins_Noise[3]*(Bin_Range_Values[y][4]+Bin_Range_Values[y][3])/2+dACG_Date_Time_Bins_Noise[4]*(Bin_Range_Values[y][5]+Bin_Range_Values[y][4])/2

        #if y=='0:30':
        #    dfx4=dfx3
        #else:
        #dfx4=pd.concat([dfx4,dfx3], axis=1, ignore_index=True)
        dACG_Date_Time_Bins_temp=np.concatenate((dACG_Date_Time_Bins_temp,dACG_Date_Time_Bins),axis=1)
        dACG_Date_Time_Bins_Noise_temp=np.concatenate((dACG_Date_Time_Bins_Noise_temp,dACG_Date_Time_Bins_Noise),axis=1)
        dACG_Date_Time_Bined_Accu_Elec_temp=np.concatenate((dACG_Date_Time_Bined_Accu_Elec_temp,dACG_Date_Time_Bined_Accu_Elec),axis=0)
        dACG_Date_Time_Bined_Noise_Accu_Elec_temp=np.concatenate((dACG_Date_Time_Bined_Noise_Accu_Elec_temp,dACG_Date_Time_Bined_Noise_Accu_Elec),axis=0)
    
    #dfx5=pd.concat([dfx5,dfx4], axis=0, ignore_index=True)

In [ ]:
dACG_Date_Time_Bined_Accu_Elec_Final=dACG_Date_Time_Bined_Accu_Elec_temp[1:].reshape(17520, 1, order='C')
dACG_Date_Time_Bined_Noise_Accu_Elec_Final=dACG_Date_Time_Bined_Noise_Accu_Elec_temp[1:].reshape(17520, 1, order='C')

dACG_Date_Time_Bined_Accu_Elec_Final.shape

In [ ]:
with open('Bins_Infos.txt', 'w') as f:
    for item in dACG_Date_Time_Bins_temp:
        f.write("%s\n" % item)

In [ ]:
with open('dACG_Date_Time_Bined_Accu_Elec_Final.txt', 'w') as f:
    for item in dACG_Date_Time_Bined_Accu_Elec_Final:
        f.write("%s\n" % item)

In [ ]:
dACGG=data_All4Postcodes_Consumption_GG
dACG_groupby_Date_Sum=dACG.groupby(['date']).sum()
dACG_groupby_Date_Sum_temp=dACG_groupby_Date_Sum.loc[:,'0:30':'0:00']
new_rows = int(dACG_groupby_Date_Sum_temp.shape[1]) * dACG_groupby_Date_Sum_temp.shape[0]
new_cols = 1
dACG_Extact_Accu_Elec_Final=dACG_groupby_Date_Sum_temp.values.reshape(new_rows, new_cols, order='C')
dACG_Extact_Accu_Elec_Final
dACGG_groupby_Date_Sum=dACGG.groupby(['date']).sum()
dACGG_groupby_Date_Sum_temp=dACGG_groupby_Date_Sum.loc[:,'0:30':'0:00']
new_rows = int(dACGG_groupby_Date_Sum_temp.shape[1]) * dACGG_groupby_Date_Sum_temp.shape[0]
new_cols = 1
dACGG_Extact_Accu_Elec_Final=dACGG_groupby_Date_Sum_temp.values.reshape(new_rows, new_cols, order='C')
dACGG_Extact_Accu_Elec_Final

In [ ]:
dACGGMGC_Date_Time_Bined_Noise_Accu_Elec_Final=dACGG_Extact_Accu_Elec_Final-dACG_Date_Time_Bined_Noise_Accu_Elec_Final
dACGGMGC_Date_Time_Bined_Accu_Elec_Final=dACGG_Extact_Accu_Elec_Final-dACG_Date_Time_Bined_Accu_Elec_Final
dACGGMGC_Extact_Accu_Elec_Final=dACGG_Extact_Accu_Elec_Final-dACG_Extact_Accu_Elec_Final
dACGGMGC_Extact_Accu_Elec_Final

In [ ]:
dataset_Bined_Noise_Accu_Elec_Final = pd.DataFrame({'Column1':dACGGMGC_Date_Time_Bined_Noise_Accu_Elec_Final[:,0]})
dataset_Bined_Accu_Elec_Final = pd.DataFrame({'Column1':dACGGMGC_Date_Time_Bined_Accu_Elec_Final[:,0]})

dataset_Extact_Accu_Elec_Final = pd.DataFrame({'Column1':dACGGMGC_Extact_Accu_Elec_Final[:,0]})
dataset_Extact_Accu_Elec_Final2 = pd.DataFrame({'Column1':dACGG_Extact_Accu_Elec_Final[:,0]})

print(dataset_Extact_Accu_Elec_Final2)


In [ ]:
hourlyTimestamp = pd.date_range(start = '2012-07-01 00:30:00', end = '2013-07-01', freq ="30min")
hourlyTimestamp
dataset_Bined_Noise_Accu_Elec_Final.index=hourlyTimestamp
dataset_Bined_Accu_Elec_Final.index=hourlyTimestamp
dataset_Extact_Accu_Elec_Final.index=hourlyTimestamp
dataset_Extact_Accu_Elec_Final2.index=hourlyTimestamp
dataset_Bined_Noise_Accu_Elec_Final

In [ ]:
dataset_Bined_Noise_Accu_Elec_Final.to_csv('dataset_Bined_Noise_Accu_Elec_Finalv8.csv')
dataset_Bined_Accu_Elec_Final.to_csv('dataset_Bined_Accu_Elec_Finalv8.csv') 
dataset_Extact_Accu_Elec_Final.to_csv('dataset_Extact_Accu_Elec_Finalv8.csv') 